# Analisi dati relazione 2, esperienza 4
_martedì 21 dicembre_


## Osservazioni sulle relazioni di laboratorio n. 1

### Osservazioni sui fit

Spesso ci affidiamo troppo al risultato del fit, trascurando l'effettivo risultato del fit, ovvero osservando i valori di $\chi^2/\text{ndf}$ e poi verificandone così la bontà.

Ipotizzziamo che il fit sia vero a priori e sia più affidabile del nostro stesso senso fisico, ignorando quindi eventuali effetti secondari che si potrebbero osservare. 

> Si può in effetti provare ad escludere alcuni punti da quelli utilizzati per esempi osservando che siano messi fuori dalla linea del fit.

### Osservazioni sul fattore di merito

Il fattore di merito $Q$ del circuito RLC si presentava come

$$
H(\omega) = \frac{R}{R+R_L + j\left(\omega L  \frac{1}{\omega C}\right)}
$$

che riscrivendo in termini di modulo diventa necessariamente

$$
|H(\omega)| = \frac{1}{\left(1+ \frac{R}{R_L}\right) + Q_{id}\left(\frac{\omega}{\omega_0}-\frac{\omega_0}{\omega}\right)}
$$

dove però il fattore di merito è un fattore di merito ideal, mantre osserviamo l'esistenza di un fattore di merito $Q_{reale}$ reale, che tiene conto delle caratteristiche di impedenza dell'induttanza, dove effettivamente il suo valore dipende da diversi fattori come

$$
Q_{reale} = \frac{\sqrt{L}}{R_{tot} \sqrt{C}}
$$

dove $R_{tot}$ dipende da tutte le resistenze presenti nel circuito.

> (Il modo di ovviare a questo problema forse lo possiamo trovare nel considerare $L_0$ come il valore di $L$ trovato non dai calcoli ma dal valore di $L_0$ misurato nel caso libero)

### Osservazioni sui risultati ottenuti

Utilizzare senso fisico per descrivere il fenomeno osservato.

Rileggendo la relazione osservo che in effetti non sono forniste esaustive giustificazioni riguardo al valore di $\mu_R \text{(Al)}$ ottenuto ($\mu_R\text{(Al)} = 0.446(53)$) non corrisponde al modello (materiale paramagnetico).

In [9]:
%jsroot on
gInterpreter->AddIncludePath("/usr/local/include");
gSystem->Load("/usr/local/lib/libErrorAnalysis");
gSystem->Load("/usr/local/lib/libLabTools");

#include<vector>
#include<cmath>
#include<iostream>
#include<fstream>
#include<string>

#include<TCanvas.h>
#include<TGraphErrors.h>
#include<TF1.h>
#include<TStyle.h>
#include<TAxis.h>
#include<TMath.h>
#include<TLatex.h>
#include<TLegend.h>

In [10]:
#include<ErrorAnalysis.h>
#include<LabTools.h>

In [11]:
// graphset::init();

std::string pfix = "../dati/_zeroed/";
const int full_dataT_s = 8;
std::string all_data_table[] = {
    "dati_2.txt",  //> 0
    "dati_4.txt",  //> 1
    "dati_5.txt",  //> 2
    "dati_6.txt",  //> 3
    "dati_8.txt",  //> 4 
    "dati_10.txt", //> 5
    "dati_12.txt", //> 6
    "dati_20.txt" //> 7
};

std::ifstream data_table;

## Commenti Carlo Schiavi su possibili modelli per analizzare le curve di riscaldamento della lampadina

Abbiamo dei dati che rappresentano delle curve di riscaldamento, dobbiamo individuare quali sono le quantità che misuriamo e quelle che possiamo individuare attraverso dei calcoli. 

#### Cosa abbiamo misurato?

Abbiamo ottenuto delle curve di dati di tensione in funzione del tempo. 

Mi sono venuti in mente più modi per scegliere il modello su come affrontare questi dati?

_Quantità da misurare?_

Ci interessa misurare una differenza di tensione $\Delta V$. 

### Considerazioni di tipo fisico

La lampadina scalda il PT100, che però prima passa attraverso il vetro, che però poi passa attraverso una bolla d'aria. 

Quantità da misurare: Variazione di tensione in funzione del tempo $\Delta V (\Delta t)$

Definisco queste qunatità come segue:
- scelgo tempo $t=0$ quando accendo la lampadina;
- scelgo $\Delta t$ tale che io possa rimanere in regime di irraggiamento

Lasciamo per ora indietro il discorso sull'incertezza.

Arbitrarietà:
- Scelta di $\Delta t$: come cambiano i risultati scegliendo un altro valore di questo intervallo?  
(NOTA: interessante potrebbe essere notare cosa succede se sceglo un $\Delta t$ con valori anche nell'intervallo del secondo regime.
- Sistematica basata sullo studio del risultato finale con diversi $\Delta t$.

### Esempio funzionale per lo studio dei modelli possibili

Procedo ad analizzare le misure effettuate ad una distanza di .05 m 

In [12]:
int table = 2; 
std::string name_data_t[8] = {"d = 2#times10^{-2}m","d = 4#times10^{-2}m",
                              "d = 5#times10^{-2}m","d = 6#times10^{-2}m"
                              "d = 8#times10^{-2}m","d = 10#times10^{-2}m",
                              "d = 12#times10^{-2}m","d = 20#times10^{-2}m"};

data_table.open((pfix + all_data_table[table]).c_str());
TCanvas *c1 = new TCanvas();
c1->SetGrid();
TGraphErrors *g = new TGraphErrors();

double t, vin, evin, range; 

for(int j=0; data_table >> t >> vin >> evin >> range; j++){
    g->SetPoint(j, t, vin);
    g->SetPointError(j, 0, 0.01);
}
g->Draw("Ap");
g->SetTitle((name_data_t[table] + ";Time [s]; #splitline{Voltage}{Serial reading [V]}").c_str());
c1->Draw();
data_table.close();

In [13]:
TCanvas *c3 = new TCanvas("","", 1000, 500);
graphset::setcanvas(c3,4,2);

for(int k=0; k<8; k++){
    ifstream *data = new ifstream((pfix + all_data_table[k]).c_str());
    TGraphErrors *g1 = new TGraphErrors();
    for(int j=0; (*data >> t >> vin >> evin >> range) && j<60; j++){
        g1->SetPoint(j, t, vin);
        g1->SetPointError(j, 0.1, 0.01);
    }
    c3->cd(k+1);
    graphset::setcanvas(c3->cd(k+1));
    g1->Draw("Ap");
    g1->GetXaxis()->SetLimits(g1->GetX()[0], g1->GetX()[0] + 80);
    g1->SetTitle((all_data_table[k] + ";Time [s]; Voltage [V]").c_str());
    data->close();
}
c3->Draw();

## Come potremmo calcolare la variazione di tensione rispetto al tempo

### Modello #1 (linearizzazione sul primo fronte di discesa)

Considerando solo i primi punti rispetto a $t=0$, preso quando viene accesa la lampadina, testare un modello lineare che prevede un andamento del tipo 

$$
V_{\text{out}}(t) = \text{(param0)} \cdot t + \text{(param1)}
$$

dove quindi possiamo individuare l'effettiva velocità $dV/dt$ con cui la tensione cambia rispetto al tempo. A questo punto poi la trattazione procede.

#### Errore sulla distanza (modello #1)

L'errore sulla distanza è legato a diversi fattori: 


| Quantità | valore |
|---|---|
| allineamento del bulbo con il supporto (poteva scivolare da una misura all'altra di qualche millimetro) | 1mm |
| allineamento del supporto con il righelllo |1mm |
| allineamento del righello con il segno sulla base del PT100 | 1mm |
| errore di parallasse per ogni allineamento | 2mm |
| errore sul possibile scivolamento durante le misure | 2mm |
| $\varepsilon_d~(\text{stat})~~~(1\sigma)$ | $7/\sqrt{3}$ mm |


L'errore sistamatico (offset) può essere dovuto a:

| Quantità | ipotesi valore |
|---|---|
| I segni per l'allineamento sulla base del PT100 potrebbero non essere perpendicolari al piano | 3mm |
| I segni per l'allineamento sulla base della lampadina e il suo supporto | 3mm |
| possibile disallineamento tra lampadina e PT100, quindi sottostima della distanza | - |
| $\varepsilon_d~(\text{sys})~~~(1\sigma)$ | $\geq6/\sqrt{3}$ mm |

#### Modello di curva per dipendenza come $d^{-2}$

Il modello utilizzato è un modello del tipo
$$
\frac{dV}{dt} = V' = \alpha\cdot(d-\gamma)^{-\beta}
$$

In [14]:
TCanvas *c4 = new TCanvas("","",1000,400);
graphset::setcanvas(c4, 2,1);
double d[8] = {0.02, 0.04, 0.05, 0.06, 0.08, 0.1, 0.12, 0.2};
double dVdt[8];
double err_dVdt[8];

Color_t mk[8] = {kRed, kOrange, kGreen-5, kAzure, kViolet-8, kGray+1, kBlue-3, kSpring-6};
//Color_t mk[8] = {100, 95, 90, 84, 65, 225, 57, 223};

c4->cd(1);
graphset::setcanvas(c4->cd(1));

TLegend *l = new TLegend(0.2, 0.2, 0.6, 0.5);
l->SetHeader("#bf{MODELLO #1}");
l->SetBorderSize(0);
l->SetFillColorAlpha(kWhite, 0);

for(int k = 0; k<8; k++){
    TGraphErrors *g1 = new TGraphErrors();
    ifstream data((pfix + all_data_table[k]).c_str());
    std::string dist = all_data_table[k].substr(all_data_table[k].find("_")+1, 
                                                all_data_table[k].find(".")-all_data_table[k].find("_")-1);

    g1->SetName(("g1_"+dist).c_str());
    
    double t1, v1, ev1, range1;

    data >> t1 >> v1 >> ev1 >> range1;

    g1->SetPoint(0, 0, 0);
    g1->SetPointError(0, 0, 0.014);
    const double t0 = t1; // zeroing data
    const double v0 = v1;

    for(int j = 1; (data >> t1 >> v1 >> ev1 >> range1) && (j<15); j++){
        
        if(k==0 && t1<16.5 && t1>15.5) continue; // tolto punto doppio per d=2cm
        if(k==2 && t1<35 && t1>34) continue;     // tolto punto doppio per d=5cm
        
        g1->SetPoint(j, t1-t0, v1-v0);
        g1->SetPointError(j, 0, 0.014);
    }
    data.close();

    TF1 *mod1 = new TF1(("mod1_"+ dist).c_str(), "[0]*x+[1]", 0, 10); // Per cambiare intervallo fit
    graphset::setmarker(g1, mod1, mk[k], 21, 0.4);
    mod1->SetParameter(0, 0.1);

    //logs::print_mmsg(("processing graph" + all_data_table[k]).c_str());
    
    g1->Draw((k==0)? "ap":"p");
    g1->SetTitle("; Time after t=0 [s]; Voltage [V]");
    graphset::set_TGraphAxis(g1, "Voltage [V]", 1, "Time after t=0 [s]");
    g1->Fit(("mod1_"+ dist).c_str(), "QR"); // fit sui primi 10s
    l->AddEntry(("g1_"+ dist).c_str(), ("d=" + dist + "#times10^{-2} m").c_str());
    g1->GetXaxis()->SetLimits(-1, 20);

    std::cout << "[[" << mod1->GetParameter(0) << " +- " << mod1->GetParError(0) << "]]" << std::endl;

    logs::print_stat(mod1);
    dVdt[k] = mod1->GetParameter(0);
    err_dVdt[k] = mod1->GetParError(0);
}
l->Draw();
std::cout << std::endl;

c4->cd(2);
graphset::setcanvas(c4->cd(2));
TGraphErrors *plot = new TGraphErrors();
for(int j=0; j<8; j++){
    //if(j==0 || j==2) continue;
    plot->SetPoint(j, d[j], abs(dVdt[j]));
    plot->SetPointError(j, 0.007/sqrt(3), err_dVdt[j]); // Stima dell'errore sulla distanza ad 1e-2 m => 1cm.
}
plot->Draw("ap");
plot->SetTitle(";Source distance [m];Voltage speed after 10s [V/s]");
graphset::set_TGraphAxis(plot, "Voltage speed after 10s [V/s]", 1, "Source distance [m]");

TF1 *fit_mod1 = new TF1("fit_mod1", "[0]*pow((x+[2]), [1])");
fit_mod1->SetParameters(0.01, 2, 0.05);


plot->Fit("fit_mod1");

logs::print_stat(fit_mod1);
std::cout << std::endl 
          << "d^( " << fit_mod1->GetParameter(1) << " +- " << fit_mod1->GetParError(1) << " )" << std::endl
          << "offset (sys): " << fit_mod1->GetParameter(2) << " +- " << fit_mod1->GetParError(2) << std::endl;

c4->Draw();

std::cout << "Compatibilità di 2: " << 
    stattools::compatible(-2, 0, fit_mod1->GetParameter(1), fit_mod1->GetParError(1)) << std::endl;
std::cout << "compatibilità di offset con stima (6/sqrt(3) mm): " <<
    stattools::compatible(6e-3, 0, fit_mod1->GetParameter(2), fit_mod1->GetParError(2)) << std::endl;

[[-0.0692979 +- 0.0014973]]

** CHI2 / NDF ( PROB. ) 7.30354 / 6 ( 0.293686 )

[[-0.0488412 +- 0.00147613]]

** CHI2 / NDF ( PROB. ) 2.46132 / 7 ( 0.929981 )

[[-0.0501754 +- 0.00146523]]

** CHI2 / NDF ( PROB. ) 8.12294 / 6 ( 0.229234 )

[[-0.0308415 +- 0.00147008]]

** CHI2 / NDF ( PROB. ) 2.96937 / 7 ( 0.887821 )

[[-0.0244878 +- 0.00147526]]

** CHI2 / NDF ( PROB. ) 1.74627 / 7 ( 0.972542 )

[[-0.0160445 +- 0.00147859]]

** CHI2 / NDF ( PROB. ) 0.68571 / 7 ( 0.998443 )

[[-0.0127483 +- 0.00148104]]

** CHI2 / NDF ( PROB. ) 0.11801 / 7 ( 0.999996 )

[[-0.00536735 +- 0.00148098]]

** CHI2 / NDF ( PROB. ) 0.00303672 / 7 ( 1 )


 FCN=9.5445 FROM HESSE     STATUS=OK             16 CALLS         474 TOTAL
                     EDM=1.1308e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           7.42911e-05   8.84530e-05   8.64382e-10  -3.3

### Modello #2 (valutazione della variazione senza modellizzare la curva)

Procediamo azzerando i valori della tensione a $t=0$ s quando la lampadina viene accesa, considerando poi un intervallo sensato (_vedere dopo_) possiamo leggere la misura della tensione, poi valutare quindi la variazione. 

Per scegliere un intervallo sensato bisogna effettuare alcuni ragionamenti di come si propaga il calore e di come quindi interagisce con il PT100, e come risponde lo strumento. 

All'accensione dalla lampadina la luce viaggia a velocità $c$ quindi possiamo di fatto considerare la sua interazione con il termometro al platino (PT100) immediata. Dopo un po' di tempo però sul il termometro interviene un secondo fattore che non permette una misura veritiera della temeperatura. Dentro alla bolla contenente il termometro infatti aria intrappolata si scalda e dopo un certo intervallo (impossibile determinarlo solo con pochi dati a disposizione) interviene e contribuisce alla variazione della temperatura letta dallo strumento. Quindi non osserviamo più solo una lettura dell'irraggiamento, ma osserviamo anche una misura della temperatura dell'ambiente circostante, che interviene in modo importante e principale dopo un certo intervallo $\Delta t_1$. misurando quindi la variazione di tensione entro $\Delta t_1$ possiamo di fatto trascurare questo fatto. Potrebbe essere interessante procedere invece ad evidenziare intervalli successivamente grandi $\Delta t_2$, $\Delta t_3$, ... per vedere come si può comportare il risultato finale. 

### Modello #3 (derivazione fine del modello #2)

Procedere in modo simile al #2 ma di fatto effettuare alcune considerazioni sugli errori ottenuti da alcune considerazioni sulla variazione effettiva della tensione sul tempo.

### Idea di modello #4 (proposta da portare a Schiavi, divisione delle componenti separando fenomeno irraggiamento e agitazione termica)

Provare a modellizzare il comportamento lineare sulla coda dei dati, quindi poi effettivamente sottrarre questo "rumore" ai valori dei dati, quindi ottenendo un comportamento che potrebbe essere di saturazione (come effettivamente ci aspettiamo di avere per il fenomeno dell'irraggiamento) e quindi osservando una curva asintotica all'infinito. 

Poi procedere simili al modello proposto in #2 o in #3. 

In [15]:
// testing in .05 m setup
TCanvas *c5 = new TCanvas();
c5->SetGrid();

g->Draw("ap");

double xmin = 65;
double xmax = 160;

TF1 *mod4_subtract = new TF1("mod4_subtract", "[0]+[1]*x", xmin, xmax);
g->Fit("mod4_subtract", "R");

TGraphErrors *clean = (TGraphErrors*)g->Clone();
graphset::setmarker(clean, kGreen-1);
for(int j=0; j<g->GetN(); j++){
    clean->SetPoint(j, g->GetX()[j], 
                    (g->GetX()[j]<=xmin)? 
                    g->GetY()[j]:g->GetY()[j]+abs(mod4_subtract->Eval(xmin)-mod4_subtract->Eval(g->GetX()[j])));
    clean->SetPointError(j, 0, 0.014);
}

clean->Draw("p");

c5->Draw()

 FCN=135.734 FROM MIGRAD    STATUS=CONVERGED      42 CALLS          43 TOTAL
                     EDM=7.80791e-20    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0          -1.53473e+00   4.99262e-03   6.59293e-06   4.31094e-08
   2  p1          -7.76223e-03   4.36132e-05   3.00000e-06  -4.18744e-06


### Considerazioni sull'incertezza

Come possiamo fare a valutare il valore di errore sulla misura di tensione?

Abbiamo raccolto un set di dati che rappresentato il valore di tensione letta quando la lampadina era spenta e quando solo la strumentazione era messa in tensione. 

Eseguendo un histo, possiamo effettivamente vedere se i dati si dispongono secondo una gausssiana, e nel caso effettivamente misurare come valore di errore a $1\sigma$ la dev. std. dell'istogramma.

In [34]:
std::string rumore = "../dati/valutazione_rumore.txt";

TCanvas *c2 = new TCanvas("","",1000, 500);
graphset::setcanvas(c2,3,1);

gStyle->SetOptStat(0);

double T, V, eV, range, crange;
std::ifstream file(rumore.c_str());
TH1D *hist = new TH1D("", "", 20, -25e-3, 35e-3);
TGraphErrors *ghist = new TGraphErrors();
for(int j=0; file >> T >> V >> eV >> range >> crange; j++){
    hist->Fill(V);
    ghist->SetPoint(j, T, V);
    ghist->SetPointError(j, 0, 0.001);
}

c2->cd(1);
graphset::setcanvas(c2->cd(1));
hist->Draw("HIST");
hist->GetYaxis()->SetRangeUser(0,250);
hist->SetTitle(";Voltage [V]; Entries");

c2->cd(2);
graphset::setcanvas(c2->cd(2));
ghist->Draw("ap");
ghist->SetTitle("Strumentazione in tensione; Time [s]; Voltage [V]");
TF1 *sin = new TF1("sin", "[0]+[2]*cos([1]*(x-[3]))");
sin->SetParameters(0.005, 0.06, 0.03, 50);
const double* params = sin->GetParameters();
ghist->Fit("sin");

c2->cd(1);

TH1D *hist_corrected = new TH1D("","",20, -25e-3, 35e-3);
hist_corrected->SetLineColor(kRed);
TFormula *sin_ev = new TFormula("","[0]+[2]*cos([1]*(x-[3]))");
sin_ev->SetParameters(params);

for(int j=0; j<ghist->GetN(); j++){
    hist_corrected->Fill(ghist->GetY()[j]-sin_ev->Eval(ghist->GetX()[j]));
}
hist_corrected->Draw("SAME");

std::cout << "RMS (pre) : " << hist->GetRMS() << " +- " << hist->GetRMSError() << std::endl
          << "RMS (post): " << hist_corrected->GetRMS() << " +- " << hist_corrected->GetRMSError() << std::endl;

c2->cd(3);
graphset::setcanvas(c2->cd(3), 2,5);
std::ifstream *batch_div = new ifstream(rumore.c_str());
int b = 0;
double xminv[10] = {0.01, 0.01, 0, 0, 0, -0.02, -0.02, -0.02, -0.01, 0};
double xmaxv[10] = {0.03, 0.03, 0.03, 0.02, 0.01, 0.01, 0, 0, 0.01, 0.02};
bool isnotlast;
do{
    TH1D *h = new TH1D(("h_" + std::to_string(b)).c_str(), "", 20, xminv[b], xmaxv[b]);
    for(int i = 0; *batch_div >> T >> V >> eV >> range >> crange && i<50; i++){
        h->Fill(V);
    }
    b++;
    c2->cd(3)->cd(b);
    h->Draw((b==0)? "HIST":"SAME");
    std::cout << "RMS (" << b << ") : " << h->GetRMS() << " +- " << h->GetRMSError() << std::endl;
}while(b!=10);

c2->Draw();

 FCN=7401.82 FROM MIGRAD    STATUS=CONVERGED     186 CALLS         187 TOTAL
                     EDM=4.46746e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           6.71177e-03   5.53611e-05   1.91956e-06  -1.47254e+00
   2  p1           5.32602e-02   1.54928e-04   2.83001e-06   3.47184e+00
   3  p2           1.79162e-02   6.36378e-05   2.63783e-06  -1.39505e+00
   4  p3           1.31560e+01   1.60596e-01   2.93281e-03  -4.16576e-03
RMS (pre) : 0.0135718 +- 0.000438486
RMS (post): 0.00393099 +- 0.000127004
RMS (1) : 0.000924724 +- 9.24724e-05
RMS (2) : 0.00106218 +- 0.000106218
RMS (3) : 0.00587398 +- 0.000587398
RMS (4) : 0.00116969 +- 0.000116969
RMS (5) : 0.000986797 +- 9.86797e-05
RMS (6) : 0.00592893 +- 0.000592893
RMS (7) : 0.00268504 +- 0.000268504
RMS (8) : 0.00132736 +- 0.000132736
RMS (9) : 0.00311095 +- 0.000311095
RMS (1

Warning in <TROOT::Append>: Replacing existing TH1: h_0 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_2 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_3 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_4 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_5 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_6 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_7 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_8 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_9 (Potential memory leak).


#### Alcune note finali

Andando ad eseguire un fit lineare sui primi 5/6 punti, ovvero fino a quando possiamo effettivamente osservare un regime lineare, posso trovare la velocità a cui il corpo si scalda. 

#### Valutazione errore

La curva in valutazione errore non presenta una guassiana perfetta. 
Potremmo procedere per batches, ovvero dividendo i punti, che sono effettivamente tanti, in batch da 40/50 punti e poi procedere a misurare RMS su questi valori, a questo punto poi verificarne la compatibilità e quindi verificare come di comportino singolarmente. 